In [13]:
import pandas as pd 
import altair as alt 
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import altair as alt
from vega_datasets import data
from plotly.subplots import make_subplots

from vega_datasets import data
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [14]:
from IPython.display import display
def display_df(df, rows=pd.options.display.max_rows, cols=pd.options.display.max_columns):
    """Displays n rows and cols from df"""
    with pd.option_context("display.max_rows", rows,
                           "display.max_columns", cols):
        display(df)

In [15]:
df = pd.read_excel('C:\\Users\\wys6299\\Desktop\\UCSD\\dsc106-wi24\\projects\\Final Project\\ukraine_conflicts\\static\\Ukraine_Black_Sea_2020_2023_Feb16.xlsx')

In [16]:
display_df(df.head(), cols=31)

,EVENT_ID_CNTY,EVENT_DATE,YEAR,TIME_PRECISION,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,INTER1,ACTOR2,ASSOC_ACTOR_2,INTER2,INTERACTION,CIVILIAN_TARGETING,ISO,REGION,COUNTRY,ADMIN1,ADMIN2,ADMIN3,LOCATION,LATITUDE,LONGITUDE,GEO_PRECISION,SOURCE,SOURCE_SCALE,NOTES,FATALITIES,TAGS,TIMESTAMP
0,ROU2045,2023-01-16,2023,1,Strategic developments,Strategic developments,Disrupted weapons use,Military Forces of Romania (2021-2023),NaN,1,Military Forces of Russia (2000-),NaN,8,18,NaN,642,Europe,Romania,Constanta,Mihai Viteazu,NaN,Canalul Periboina,44.611,28.929,2,Adevarul; News.ro,National,"Defusal: On 16 January 2023, Romanian Naval Fo...",0,NaN,1687286172
1,ROU2026,2022-12-10,2022,1,Strategic developments,Strategic developments,Disrupted weapons use,Military Forces of Romania (2021-2023),NaN,1,Unidentified Military Forces,NaN,8,18,NaN,642,Europe,Romania,Constanta,NaN,NaN,Coast of Constanta,44.156,28.948,1,Agerpres; Deschide,National-Regional,"Defusal: On 10 December 2022, the Romanian nav...",0,NaN,1687286172
2,ROU1961,2022-09-08,2022,1,Political violence,Explosions/Remote violence,Remote explosive/landmine/IED,Unidentified Military Forces,NaN,8,Military Forces of Romania (2021-2023),NaN,1,18,NaN,642,Europe,Romania,Constanta,NaN,NaN,Coast of Constanta,44.156,28.948,1,Adevarul; Balkan Insight; Digi24; G4media; RFE/RL,National-Regional,"On 8 September 2022, a sea mine of unknown ori...",0,NaN,1687286172
3,ROU1947,2022-08-04,2022,1,Demonstrations,Protests,Peaceful protest,Protesters (Romania),Greenpeace,6,NaN,NaN,0,60,NaN,642,Europe,Romania,Constanta,NaN,NaN,Coast of Constanta,44.156,28.948,1,News.ro,National,"On 4 August 2022, Greenpeace activists protest...",0,crowd size=no report,1660055882
4,ROU1945,2022-07-31,2022,1,Strategic developments,Strategic developments,Disrupted weapons use,Military Forces of Romania (2021-2023),NaN,1,Unidentified Armed Group (International),NaN,3,13,NaN,642,Europe,Romania,Constanta,NaN,NaN,Coast of Constanta,44.156,28.948,1,Digi24,National,"Defusal: On 31 July 2022, Romanian Naval Force...",0,NaN,1687286172


In [17]:
# Define the bin sizes for latitude and longitude
lat_bins = [min(df['LATITUDE']), 50, 51, 52, max(df['LATITUDE'])]  # Define latitude bins
lon_bins = [min(df['LONGITUDE']), 33, 34, 35, max(df['LONGITUDE'])]  # Define longitude bins

# Define labels for the bins
lat_labels = ['bin1', 'bin2', 'bin3', 'bin4']
lon_labels = ['bin1', 'bin2', 'bin3', 'bin4']

# Add bin columns based on latitude and longitude
df['LAT_BIN'] = pd.cut(df['LATITUDE'], bins=lat_bins, labels=lat_labels)
df['LON_BIN'] = pd.cut(df['LONGITUDE'], bins=lon_bins, labels=lon_labels)

# Optional: You can combine the latitude and longitude bins into a single column if needed
df['BIN'] = df['LAT_BIN'] + '-' + df['LON_BIN']

# Display the updated DataFrame
print(df)

TypeError: unsupported operand type(s) for +: 'Categorical' and 'str'

In [6]:
ukraine = df[(df['YEAR'] >= 2021) & (df['COUNTRY'] >= 'Ukraine') & (df['TIME_PRECISION'] >= 1) & (df['GEO_PRECISION'] >= 2)]
ukraine = ukraine[['EVENT_DATE', 'YEAR', 'DISORDER_TYPE', 'EVENT_TYPE', 'SUB_EVENT_TYPE', 'ACTOR1', 'LOCATION', 'LATITUDE', 'LONGITUDE', 'NOTES', 'FATALITIES']].reset_index().drop('index', axis=1)
print(ukraine.head().to_markdown(index=False))

| EVENT_DATE          |   YEAR | DISORDER_TYPE      | EVENT_TYPE                 | SUB_EVENT_TYPE   | ACTOR1                                      | LOCATION    |   LATITUDE |   LONGITUDE | NOTES                                                                                                                                                                                                                                                                                                                                    |   FATALITIES |
|:--------------------|-------:|:-------------------|:---------------------------|:-----------------|:--------------------------------------------|:------------|-----------:|------------:|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
ukraine.to_csv('filtered.csv', index=False)

In [8]:
ukraine['EVENT_TYPE'].value_counts()

EVENT_TYPE
Explosions/Remote violence    44490
Battles                       18010
Strategic developments         5935
Violence against civilians      220
Protests                         29
Name: count, dtype: int64

In [9]:
# ukraine[ukraine['EVENT_TYPE'] == 'Protests'].to_excel('protests.xlsx')
# ukraine[ukraine['EVENT_TYPE'] == 'Battles'].to_excel('battles.xlsx')
# ukraine[ukraine['EVENT_TYPE'] == 'Explosions/Remote violence'].to_excel('explosion.xlsx')
# ukraine[ukraine['EVENT_TYPE'] == 'Strategic developments'].to_excel('develop.xlsx')
# ukraine[ukraine['EVENT_TYPE'] == 'Violence against civilians'].to_excel('violance.xlsx')